In [17]:
import pandas as pd
from pandas import DataFrame

gene2go = pd.read_csv("../Datasets/NCBI/gene2go/gene2go" , "\t")
gene2go = gene2go.loc[gene2go['#tax_id'] == 9606]
gene2go = gene2go.drop(columns=['#tax_id'])
#humanGenome = pd.read_csv("../Datasets/NCBI/Homo_sapiens.gene_info/Homo_sapiens.gene_info", "\t")
gene2go

,GeneID,GO_ID,Evidence,Qualifier,GO_term,PubMed,Category
575368,1,GO:0002576,TAS,-,platelet degranulation,-,Process
575369,1,GO:0003674,ND,-,molecular_function,-,Function
575370,1,GO:0005576,IDA,-,extracellular region,3458201|27068509,Component
575371,1,GO:0005576,TAS,-,extracellular region,-,Component
575372,1,GO:0005615,HDA,-,extracellular space,16502470,Component
575373,1,GO:0008150,ND,-,biological_process,-,Process
575374,1,GO:0031012,IDA,-,extracellular matrix,27559042,Component
575375,1,GO:0031093,TAS,-,platelet alpha granule lumen,-,Component
575376,1,GO:0034774,TAS,-,secretory granule lumen,-,Component
575377,1,GO:0043312,TAS,-,neutrophil degranulation,-,Process


In [18]:
# Generating the scripts

# (1) name : name of the table
# (2) lines : list of all lines to be inserted
# e.g. generateInsertScript("users", [["1", "25", "hello"], ['2', '3', '4']])
def generateInsertScript(name, lines):
    query = "INSERT INTO TABLE " + name  + " VALUES"
    for line in lines:
        first = True
        query = query + " ("
        for v in line:
            if first:
                first = False
            else:
                query = query + ", "
            query = query + str(v)
        query = query + ")"
    if len(lines) == 0:
        return ";"
    else:
        return query + ";"
generateInsertScript("users", [("1", "25", "hello"), (2, 3, 4)])

'INSERT INTO TABLE users VALUES (1, 25, hello) (2, 3, 4);'

In [19]:
# return a dataframe's columns named after array 'attributes'
# inefficient, rather slow
def extractColumns(df, attributes):
    return [df[attributes].iloc[[i]].values.tolist()[0] for i in range(len(df))]

# split a string and remove the empty elements
def split(s, delimiter):
    content = s.split(delimiter)
    return [item for item in content if item]


## Parsing UnitProt

In [20]:
cancer_lines = []
allowedCategories = ['ID', 'AC', 'DE', 'GN', 'KW']
with open("../Datasets/UniProtKB/unitprot-cancer/unitprot-cancer.txt") as cf:
    for line in cf:
        content = split(line, ' ')
        if len(content) > 0 and content[0] in allowedCategories:
            # 1. separate line prefix from content
            # 2. remove '\n' character at the end of every line (supposedly, otherwise don't do it like this)
            cancer_lines.append((content[0], (' '.join(content[1:]))[:-1]))

In [21]:
cancer_lines

[('ID', 'P53_HUMAN Reviewed; 393 AA.'),
 ('AC', 'P04637; Q15086; Q15087; Q15088; Q16535; Q16807; Q16808; Q16809;'),
 ('AC', 'Q16810; Q16811; Q16848; Q2XN98; Q3LRW1; Q3LRW2; Q3LRW3; Q3LRW4;'),
 ('AC', 'Q3LRW5; Q86UG1; Q8J016; Q99659; Q9BTM4; Q9HAQ8; Q9NP68; Q9NPJ2;'),
 ('AC', 'Q9NZD0; Q9UBI2; Q9UQ61;'),
 ('DE', 'RecName: Full=Cellular tumor antigen p53;'),
 ('DE', 'AltName: Full=Antigen NY-CO-13;'),
 ('DE', 'AltName: Full=Phosphoprotein p53;'),
 ('DE', 'AltName: Full=Tumor suppressor p53;'),
 ('GN', 'Name=TP53; Synonyms=P53;'),
 ('KW', '3D-structure; Acetylation; Activator; Alternative promoter usage;'),
 ('KW', 'Alternative splicing; Apoptosis; Biological rhythms; Cell cycle;'),
 ('KW', 'Complete proteome; Cytoplasm; Disease mutation; DNA-binding;'),
 ('KW', 'Endoplasmic reticulum; Glycoprotein; Host-virus interaction;'),
 ('KW', 'Isopeptide bond; Li-Fraumeni syndrome; Metal-binding; Methylation;'),
 ('KW', 'Mitochondrion; Necrosis; Nucleus; Phosphoprotein; Polymorphism;'),
 ('KW', 'Re

In [22]:
# For every type of line, specific parsing is required
# You should search for the specific syntax and parsing process on unitprot
class CancerDataNode:
     def __init__(self, id):
        self.id = id
        self.ac = []
        self.alt_names = []
        self.rec_names = []
        self.keywords = []
        self.flags = []
        self.gn = None

cancerData = []
currentNode = None
            
for line in cancer_lines:
    if not line[1]:
        continue
    form = line[0]
    if (form == 'ID'):
        currentNode = CancerDataNode(line[1].split(' ')[0])
        cancerData.append(currentNode)
    elif (form == 'AC'):
        if currentNode:
            content = split(line[1], ';')
            for item in content:
                currentNode.ac.append(item)
    elif (form == 'DE'):
        #if currentNode:
        #    spacesplit = split(line[1], ' ')
        #    eqsplit = split(line[1], '=')
        #    if len(spacesplit) > 1:
        #        # AltName: Full=$name
        #        # RecName: Full=$name
        #        # Name: Full=$name
        #        if ':' in spacesplit[0]:
        #            currentNode.addDE(spacesplit[0], split(' '.join(spacesplit[:-1]), '=')[1])
        #    elif len(spacesplit) == 1:
        #        
        #        
        #    currentNode.alt_names.append(line[1].split(':')[1].strip())
        pass
    elif (form == 'GN'):
        pass
    elif (form == 'KW'):
        pass
    
for t in cancerData:
    print(t.id)


P53_HUMAN
BRCA1_HUMAN
ANDR_HUMAN
EGFR_HUMAN
BRCA2_HUMAN
CTNB1_HUMAN
AKT1_HUMAN
PTEN_HUMAN
LRRK2_HUMAN
SIR1_HUMAN
TITIN_HUMAN
MUC1_HUMAN
GOGA2_HUMAN
MLH1_HUMAN
ATM_HUMAN
TGFB1_HUMAN
RACK1_HUMAN
PRKN_HUMAN
ERBB2_HUMAN
FGFR2_HUMAN
EP300_HUMAN
NRG1_HUMAN
MSH2_HUMAN
PSN1_HUMAN
KCNH2_HUMAN
SMAD3_HUMAN
1433E_HUMAN
3BP1_HUMAN
AA1R_HUMAN
AA3R_HUMAN
ACSL5_HUMAN
ACTL8_HUMAN
ACTN1_HUMAN
ACVR1_HUMAN
ACY1_HUMAN
ADA10_HUMAN
ADA17_HUMAN
ADA19_HUMAN
ADA29_HUMAN
ADAT3_HUMAN
ADA_HUMAN
ADCK2_HUMAN
ADCK5_HUMAN
ADIRF_HUMAN
ADPRM_HUMAN
ADX_HUMAN
AFAD_HUMAN
1433B_HUMAN
5HT3C_HUMAN
5HT5A_HUMAN
5NTC_HUMAN
AATC_HUMAN
AATF_HUMAN
ABCA2_HUMAN
ABCA3_HUMAN
ABCB8_HUMAN
ABCE1_HUMAN
ABL2_HUMAN
ACAP1_HUMAN
ACATN_HUMAN
ACD_HUMAN
ACHA5_HUMAN
ACINU_HUMAN
ACOD_HUMAN
ACSF4_HUMAN
2A5D_HUMAN
2A5E_HUMAN
2A5G_HUMAN
AAAD_HUMAN
AAK1_HUMAN
AASD1_HUMAN
ABCA1_HUMAN
ABCBA_HUMAN
ABTB1_HUMAN
ACKR1_HUMAN
ACKR4_HUMAN
ACRBP_HUMAN
ACSA_HUMAN
ACSL3_HUMAN
ACSL4_HUMAN
ACTB_HUMAN
ACTG_HUMAN
ACTL9_HUMAN
ACTN4_HUMAN
ADAM9_HUMAN
ADAS_HUMAN
ACHA3_HU

DCP1A_HUMAN
DCR1A_HUMAN
DCTN1_HUMAN
DDB1_HUMAN
DDHD1_HUMAN
DDIAS_HUMAN
DDIT3_HUMAN
DDR1_HUMAN
DDX17_HUMAN
DDX1_HUMAN
DDX21_HUMAN
DDX54_HUMAN
DEF1_HUMAN
DEP1B_HUMAN
DHI1_HUMAN
DHRS2_HUMAN
DHX15_HUMAN
DMXL2_HUMAN
DBND1_HUMAN
DCNL3_HUMAN
DCP2_HUMAN
DCTD_HUMAN
DCTN2_HUMAN
DCTP1_HUMAN
DDA1_HUMAN
DDAH1_HUMAN
DDX47_HUMAN
DDX50_HUMAN
DDX56_HUMAN
DEFM_HUMAN
DEN1B_HUMAN
DEN4B_HUMAN
DEN5A_HUMAN
DEPD5_HUMAN
DEPP1_HUMAN
DG2L6_HUMAN
DGCR8_HUMAN
DGKE_HUMAN
DGKZ_HUMAN
DGLB_HUMAN
DHB4_HUMAN
DHSD_HUMAN
DHX16_HUMAN
DHX36_HUMAN
DI3L1_HUMAN
DICER_HUMAN
DIEXF_HUMAN
DIP2A_HUMAN
DIRC1_HUMAN
DIRC2_HUMAN
DKC1_HUMAN
DKK1_HUMAN
DKK4_HUMAN
DLG2_HUMAN
DLGP4_HUMAN
DNJB9_HUMAN
DNJC7_HUMAN
DNJC9_HUMAN
DNLI1_HUMAN
DNSL3_HUMAN
DOCK2_HUMAN
DOCK4_HUMAN
DONS_HUMAN
DOP1_HUMAN
DPH2_HUMAN
DMBT1_HUMAN
DNJB6_HUMAN
DNM1L_HUMAN
DNM3A_HUMAN
DNMBP_HUMAN
DOCK1_HUMAN
DOCK6_HUMAN
DOCK8_HUMAN
DOCK9_HUMAN
DOP2_HUMAN
DPOD1_HUMAN
DPOD2_HUMAN
DPOE1_HUMAN
DPOLM_HUMAN
DPYL5_HUMAN
DHH_HUMAN
DIAP1_HUMAN
DIAP3_HUMAN
DIDO1_HUMAN
DJC12_HUMAN
DJC1

NIN_HUMAN
NIT2_HUMAN
NJMU_HUMAN
NKRF_HUMAN
NLTP_HUMAN
NMNA2_HUMAN
NOP58_HUMAN
NDUBA_HUMAN
NDUF4_HUMAN
NECA3_HUMAN
NECP1_HUMAN
NEDD1_HUMAN
NEK10_HUMAN
NEK1_HUMAN
NEK2_HUMAN
NEK6_HUMAN
NEMF_HUMAN
NEU2_HUMAN
NEUFC_HUMAN
NEUL_HUMAN
NEUR3_HUMAN
NEXN_HUMAN
NF2IP_HUMAN
NFAC1_HUMAN
NFAC2_HUMAN
NFAC3_HUMAN
NFAC4_HUMAN
NFKB1_HUMAN
NH2L1_HUMAN
NHRF2_HUMAN
NHSL1_HUMAN
NOA1_HUMAN
NOB1_HUMAN
NOC2L_HUMAN
NOL10_HUMAN
NOL6_HUMAN
NOLC1_HUMAN
NOM1_HUMAN
NOP16_HUMAN
NOP53_HUMAN
NOXA1_HUMAN
NP24_HUMAN
NR2C1_HUMAN
NRARP_HUMAN
NGAL_HUMAN
NIBAN_HUMAN
NID1_HUMAN
NID2_HUMAN
NLGNX_HUMAN
NLK_HUMAN
NLRP9_HUMAN
NMD3_HUMAN
NMDE4_HUMAN
NMES1_HUMAN
NODAL_HUMAN
NOE2_HUMAN
NOG1_HUMAN
NOS2_HUMAN
NOSIP_HUMAN
NPBW1_HUMAN
NPDC1_HUMAN
NPT2B_HUMAN
NRBP_HUMAN
NT5D3_HUMAN
NTRK1_HUMAN
NU205_HUMAN
NU214_HUMAN
NUDC2_HUMAN
NUFP1_HUMAN
NUFP2_HUMAN
NUP50_HUMAN
NUP53_HUMAN
NUP98_HUMAN
NUTM1_HUMAN
NXF1_HUMAN
NXF2_HUMAN
NXPE4_HUMAN
O51E2_HUMAN
NKD1_HUMAN
NKX31_HUMAN
NLE1_HUMAN
NLRP1_HUMAN
NMI_HUMAN
NMNA1_HUMAN
NOL7_HUMAN
NOL8_HUMAN
NOP2

SUV92_HUMAN
SYAC_HUMAN
SYAP1_HUMAN
SYCE1_HUMAN
SYDC_HUMAN
SYDE1_HUMAN
SYDE2_HUMAN
SYEP_HUMAN
SYHM_HUMAN
SYNRG_HUMAN
SYPM_HUMAN
SYT9_HUMAN
SYYC_HUMAN
T22D4_HUMAN
TAB2_HUMAN
TAF10_HUMAN
TAF4B_HUMAN
TAF6_HUMAN
TAGL2_HUMAN
SYDM_HUMAN
SYNJ1_HUMAN
SYNJ2_HUMAN
SYNP2_HUMAN
SYQ_HUMAN
SYTC2_HUMAN
SYTC_HUMAN
SYVC_HUMAN
SYWC_HUMAN
SSX1_HUMAN
SSX7_HUMAN
ST14_HUMAN
ST17B_HUMAN
ST32A_HUMAN
ST32B_HUMAN
ST5_HUMAN
STA13_HUMAN
STAT3_HUMAN
STAT4_HUMAN
STAU1_HUMAN
STAU2_HUMAN
STEA2_HUMAN
STIL_HUMAN
STK10_HUMAN
STK16_HUMAN
STK36_HUMAN
STK3_HUMAN
STMN1_HUMAN
STPG1_HUMAN
STRBP_HUMAN
STRP1_HUMAN
STRP2_HUMAN
STX5_HUMAN
SUCB1_HUMAN
SUCO_HUMAN
SULF1_HUMAN
SURF2_HUMAN
SVIL_HUMAN
SVIP_HUMAN
SYBU_HUMAN
SYMC_HUMAN
SYMPK_HUMAN
SYNC_HUMAN
SYNEM_HUMAN
SYNPO_HUMAN
SYRC_HUMAN
T2AG_HUMAN
T2FA_HUMAN
T53I2_HUMAN
TAB1_HUMAN
TAB3_HUMAN
TANC1_HUMAN
TANC2_HUMAN
TASOR_HUMAN
TAU_HUMAN
TB10B_HUMAN
TB15A_HUMAN
TBC14_HUMAN
TBC15_HUMAN
TBC25_HUMAN
TBC9B_HUMAN
TBCD5_HUMAN
TBD2B_HUMAN
TBRG1_HUMAN
TBX15_HUMAN
TET1_HUMAN
TEX14_HUMAN
TF7L1